In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'



In [51]:
words = open('input.txt', 'r').read().splitlines()
len(words)

32033

In [52]:
chars = sorted(list(set(''.join(words))))
stoi = { ch : i + 1 for i, ch in enumerate(chars) }
stoi['.'] = 0
itos = { i : ch for ch, i in stoi.items() }
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [53]:
block_size = 3
x, y = [], []

for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        x.append(context)
        y.append(ix)
        context = context[1:] + [ix]
X = torch.tensor(x).to(device)
Y = torch.tensor(y).to(device)

In [124]:
C = torch.randn((27, 2)).to(device)
W1 = torch.randn((6, 100)).to(device)
b1 = torch.randn((100)).to(device)
W2 = torch.randn((100, 27)).to(device)
b2 = torch.randn(27).to(device)
if (X >=  27).any():
    print('X has invalid values')
parameters = [C, W1, b1, W2, b2]

In [66]:
sum(p.nelement() for p in parameters)

3481

In [125]:
for p in parameters:
    p.requires_grad = True

lrs = torch.linspace(-3, 0, 1000)
lrs = 10 ** lrs
lrs

tensor([0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012,
        0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0013, 0.0013, 0.0013,
        0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0014,
        0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014,
        0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015,
        0.0015, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016,
        0.0016, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017,
        0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0019,
        0.0019, 0.0019, 0.0019, 0.0019, 0.0019, 0.0019, 0.0019, 0.0020, 0.0020,
        0.0020, 0.0020, 0.0020, 0.0020, 0.0020, 0.0021, 0.0021, 0.0021, 0.0021,
        0.0021, 0.0021, 0.0021, 0.0022, 

In [126]:
# Forward pass
lri = []
losses = []
for i in range (1000):
    ix = torch.randint(0, X.shape[0], (32,))

    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix])
    print(loss.item())
    for p in parameters:
        p.grad = None
    loss.backward()
    lr = lrs[i]
    for p in parameters:
        p.data += -lr * p.grad
    lri.append(lr)
    losses.append(loss.item())


19.09952735900879
19.826337814331055
16.34396743774414
16.54367446899414
19.02713966369629
16.43951416015625
18.317739486694336
15.165586471557617
16.49733543395996
16.294933319091797
17.042457580566406
16.71378517150879
18.896469116210938
15.523374557495117
20.60460090637207
16.617616653442383
15.789264678955078
18.29132652282715
14.235158920288086
17.983457565307617
19.31673812866211
15.161628723144531
17.380910873413086
17.068336486816406
17.89537239074707
14.389394760131836
13.497208595275879
14.810447692871094
17.684865951538086
17.030838012695312
13.402313232421875
15.141157150268555
13.582036018371582
16.813264846801758
13.831100463867188
14.430244445800781
15.902687072753906
14.196174621582031
14.496363639831543
13.889389038085938
13.189925193786621
15.511943817138672
15.45988655090332
15.213436126708984
17.731428146362305
15.852775573730469
17.506494522094727
16.975175857543945
12.953835487365723
12.829984664916992
16.856830596923828
14.889663696289062
16.45538330078125
13.790

In [63]:
emb= C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss.item()

2.517119884490967